In [1]:
import pandas as pd
import ast

In [2]:
%%time
# Assemble individual SBIR CSV files

df_list = []

file_list = ['../preprocessed_files/sbir_entities_1.csv',
             '../preprocessed_files/sbir_entities_2.csv',
             '../preprocessed_files/sbir_entities_3.csv',
             '../preprocessed_files/sbir_entities_4.csv',
             '../preprocessed_files/sbir_entities_5.csv',
             '../preprocessed_files/sbir_entities_6.csv',
             '../preprocessed_files/sbir_entities_7.csv',
             '../preprocessed_files/sbir_entities_8.csv'
            ]
for f in file_list:
    df = pd.read_csv(f)
    df_list.append(df)
sbir_df = pd.concat(df_list,ignore_index=True)

<timed exec>:15: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:15: DtypeWarning: Columns (10,28) have mixed types. Specify dtype option on import or set low_memory=False.
<timed exec>:15: DtypeWarning: Columns (8,9,12,13,14,15,28,32,36) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 3.58 s, sys: 694 ms, total: 4.27 s
Wall time: 5.19 s


In [5]:
sbir_df = sbir_df.sample(1000)

In [7]:
sbir_df.head(5)

,Company,Award Title,Agency,Branch,Phase,Program,Agency Tracking Number,Contract,Proposal Award Date,Contract End Date,...,Contact Phone,Contact Email,PI Name,PI Title,PI Phone,PI Email,RI Name,RI POC Name,RI POC Phone,abstract_entities
150881,"Mellwood Laboratorytories,",High-Power Pulsed Laser Diode Driver Utilizing...,Department of Defense,Air Force,Phase I,SBIR,28130,NaN,NaN,NaN,...,() -,NaN,Eric S. Harmon,NaN,(317) 426-3662,NaN,NaN,NaN,NaN,"['switch', 'high-power', 'power driver', 'elec..."
159895,Crystallume/edi,A SINGLE CRYSTAL DIAMOND ULTRAVIOLET DETECTOR ...,Department of Energy,NaN,Phase I,SBIR,14551,NaN,NaN,NaN,...,(415) 324-9681,NaN,Maurice Landstrass,Principal Investigator,() -,NaN,NaN,NaN,NaN,"['detectors TESTED', 'use uv laser', 'phase re..."
161341,Sunburst Recovery Inc,ELECTROMAGNETIC GAGE MEASUREMENT OF COUPLING I...,Department of Defense,Defense Advanced Research Projects Agency,Phase I,SBIR,15254,NaN,NaN,NaN,...,() -,NaN,Morris Young,Principal Investigator,(415) 833-0553,NaN,NaN,NaN,NaN,"['electromagnetic gage', 'benefits/potential',..."
155588,INFINIA CORPORATION,A STIRLING CRYOCOOLER FOR MRI CRYOSTAT SHIELD ...,Department of Health and Human Services,NaN,Phase II,SBIR,19258,NaN,NaN,NaN,...,() -,NaN,Peter Riggle,NaN,(509) 375-4000,NaN,NaN,NaN,NaN,"['LINEAR MOTOR DRIVEN COMPRESSOR', 'cost', 'FU..."
106795,"BANYAN BIOMARKERS, INC.",Novel Biomarkers for Stroke Injury,Department of Health and Human Services,National Institutes of Health,Phase I,SBIR,NS051039,1R43NS051039-01,NaN,NaN,...,(386) 462-6699,cellis@banyanbio.com,MONIKA OLI,NaN,(386) 418-1632,MOLI@BANYANBIO.COM,NaN,NaN,NaN,"['event', 'information', 'detection', 'recover..."


In [8]:
patent_df = pd.read_csv('../preprocessed_files/patents_entities.csv')

In [9]:
patent_df = patent_df.sample(1000)

In [12]:
patent_df = patent_df.set_index('doc-number')

In [13]:
%%time
sbir_df['abstract_entities'] = sbir_df['abstract_entities'].apply(lambda x:ast.literal_eval(x))
patent_df['abstract_entities'] = patent_df['abstract_entities'].apply(lambda x:ast.literal_eval(x))
sbir_df['abstract_entities'] = sbir_df['abstract_entities'].apply(lambda x:' '.join(x))
patent_df['abstract_entities'] = patent_df['abstract_entities'].apply(lambda x:' '.join(x))

CPU times: user 171 ms, sys: 13.2 ms, total: 184 ms
Wall time: 1.01 s


In [24]:
%%time
import spacy
similar_entities=[]
nlp = spacy.load("en_core_sci_lg")  # make sure to use larger package!
for i in sbir_df.index:
    doc1 = nlp(sbir_df['abstract_entities'].loc[i])
    for j in patent_df.index:
        doc2 = nlp(patent_df['abstract_entities'].loc[j])
        similarity = doc1.similarity(doc2)
        #if similarity > 0.85:
        similar_entities.append({'sbir_id':i,'patent_id':j,'sim_score':similarity})
        if similarity > 0.8:
            print({'sbir_id':i,'patent_id':j,'sim_score':similarity})

{'sbir_id': 150881, 'patent_id': 20230231582, 'sim_score': 0.8112722510865018}
{'sbir_id': 150881, 'patent_id': 20230229456, 'sim_score': 0.8005486151854132}


<timed exec>:8: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


{'sbir_id': 150881, 'patent_id': 20230231359, 'sim_score': 0.8203030911057602}
{'sbir_id': 150881, 'patent_id': 20230232514, 'sim_score': 0.8634084770412698}
{'sbir_id': 150881, 'patent_id': 20230228943, 'sim_score': 0.8022727894179007}
{'sbir_id': 150881, 'patent_id': 20230230542, 'sim_score': 0.8347567391711762}
{'sbir_id': 144190, 'patent_id': 20230229542, 'sim_score': 0.8018066219393061}
{'sbir_id': 144190, 'patent_id': 20230229282, 'sim_score': 0.8034987913659725}
{'sbir_id': 21486, 'patent_id': 20230229282, 'sim_score': 0.8052654860450585}
{'sbir_id': 17208, 'patent_id': 20230231866, 'sim_score': 0.8329640481825957}
{'sbir_id': 17208, 'patent_id': 20230229940, 'sim_score': 0.8213753422930282}
{'sbir_id': 17208, 'patent_id': 20230230190, 'sim_score': 0.8296791781774684}
{'sbir_id': 60213, 'patent_id': 20230230033, 'sim_score': 0.8371822805237228}
{'sbir_id': 60213, 'patent_id': 20230230198, 'sim_score': 0.8386838218847884}
{'sbir_id': 60213, 'patent_id': 20230228624, 'sim_score': 

In [25]:
len(similar_entities)

1000000

In [26]:
sbir_df['abstract_entities'].loc[39893],patent_df['abstract_entities'].loc[20230226687]

('efficiency system information temperature cost Efficiency Monitor building manager multi-sensor module flow network model simulation calculate envelope leakage duct humidity performance measurement high-resolution absolute differential pressure datum information    base approach wirelessly internet wirelessly system microelectromechanical system pressure',
 'electronic device information robot parameter algorithm calculate value robot motion wearable device wearable device base robot log information regard value robot wearable device base log')

In [27]:
sbir_df['abstract_entities'].loc[39893],patent_df['abstract_entities'].loc[20230225798]

('efficiency system information temperature cost Efficiency Monitor building manager multi-sensor module flow network model simulation calculate envelope leakage duct humidity performance measurement high-resolution absolute differential pressure datum information    base approach wirelessly internet wirelessly system microelectromechanical system pressure',
 'object retention Systems system use method apparatus wand system motion user interface wand body scanner network interface use system motion sensor')

In [43]:
sbir_df.to_csv('../preprocessed_files/sbir_1k_sample.csv')
patent_df.to_csv('../preprocessed_files/patent_1k_sample.csv')

In [44]:
similarity_df = pd.DataFrame(similar_entities)

In [46]:
similarity_df.to_csv('../preprocessed_files/similarity.csv')